In [5]:
import sys, os
sys.path.append("../..")

# import gempy
import gempy as gp

# inline figures in jupyter notebooks
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pn
from importlib import reload

In [2]:
import gempy.utils.geomodeller_integration as gi

In [3]:
gi

<module 'gempy.utils.geomodeller_integration' from '../..\\gempy\\utils\\geomodeller_integration.py'>

In [4]:
geoM = gi.GeomodellerClass()

In [5]:
geoM.load_geomodeller_file('../../test/GeoModeller/test_d/test_d.xml')

In [6]:
geoM.get_sections()

[<Element '{http://www.geomodeller.com/geo}Section' at 0x0000017568B94868>,
 <Element '{http://www.geomodeller.com/geo}Section' at 0x0000017568B9F188>]

In [7]:
geoM.get_faults()

[<Element '{http://www.geomodeller.com/geo}Fault' at 0x0000017568B94638>]

In [8]:
geoM.faults[0].get("Name")

'f1'

In [9]:
geoM.get_formations()

In [10]:
geoM.formations

[<Element '{http://www.geomodeller.com/geo}Formation' at 0x0000017568B789F8>,
 <Element '{http://www.geomodeller.com/geo}Formation' at 0x0000017568B8C3B8>]

In [11]:
geoM.get_formation_names()

['A', 'B']

In [12]:
geoM.get_fault_names

<bound method GeomodellerClass.get_fault_names of <gempy.utils.geomodeller_integration.GeomodellerClass object at 0x0000017568A23B70>>

In [13]:
geoM.get_stratigraphy_list()

['B', 'A']

In [14]:
geoM.get_fault_parameters()

AttributeError: 'GeomodellerClass' object has no attribute 'get_fault_parameters'

In [15]:
geoM.create_formation_dict()

{'A': <Element '{http://www.geomodeller.com/geo}Formation' at 0x0000017568B789F8>,
 'B': <Element '{http://www.geomodeller.com/geo}Formation' at 0x0000017568B8C3B8>}

In [22]:
geoM.create_fault_dict()

{'f1': <Element '{http://www.geomodeller.com/geo}Fault' at 0x7fa938261818>}

In [23]:
geoM.get_model_extent()

(0.0, 10.0, 0.0, 10.0, -10.0, 0.0)

In [31]:
geoM.rootelement.find("{"+geoM.xmlns+"}GeologicalModel").find("{"+geoM.xmlns+"}Extent3D").find("{"+geoM.xmlns+"}ExtentZ").get("Zmin")

'-10.0'

In [33]:
stc =geoM.rootelement.find("{"+geoM.xmlns+"}GeologicalModel").find("{"+geoM.xmlns+"}ProjectStratigraphicColumn")

In [35]:
stc.find("{"+geoM.gml+"}coord")

In [40]:
inter = stc.find("{"+geoM.xmlns+"}Series").find("{"+geoM.xmlns+"}PotentialField").find("{"+geoM.xmlns+"}Points")

In [50]:
p = inter.find("{"+geoM.gml+"}coord").findtext("{"+geoM.gml+"}X")

In [49]:
p.text

'2.0477319587628866059958454570733010768890380859375'

In [51]:
p

'2.0477319587628866059958454570733010768890380859375'

---

In [284]:
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET
    

class ReadGeoModellerXML:
    def __init__(self, fp):
        self.tree = ET.ElementTree(file=fp)  # load xml as tree
        self.root = self.tree.getroot()
        
        self.extent = self._get_extent()
        self.series = self._get_series()
        
    def _get_extent(self):
        xy = root[0][0][0][0].attrib
        z = root[0][0][0][1].attrib
        return tuple(np.array([xy["Xmin"], xy["Xmax"], 
                               xy["Ymin"], xy["Ymax"], 
                               z["Zmin"], z["Zmax"]]).astype(float))

    def _get_series(self):
        return [s.attrib["name"] for s in self.root[6][3]]
    
    def _get_series_interf_iter(self, n_series):
        """Returns Points element for given series number, over which you 
        can iterate to get the actual point coordinates."""
        return self.root[6][3][n_series][-1][2]
    
    def _get_series_interf_coords(self, n_series):
        points = self._get_series_interf_iter(n_series)
        for i, p in enumerate(points):
            if i == 0:
                xyz = np.array([[points[0][0].text, points[0][1].text, points[0][2].text]]).astype(float)
            else:
                xyz = np.append(xyz, np.array([[points[0][0].text, points[0][1].text, points[0][2].text]]).astype(float), axis=0)

        return xyz
    
ReadGeoModellerXML("NorthPerthBasin_rev.xml")._get_series_interf_coords(6).shape

(1541, 3)

In [67]:
reload(gp.utils.geomodeller_integration)

<module 'gempy.utils.geomodeller_integration' from '../..\\gempy\\utils\\geomodeller_integration.py'>

In [68]:
gmx = gp.utils.geomodeller_integration.ReadGeoModellerXML("NorthPerthBasin_rev.xml")

In [65]:
df = gmx.get_interfaces()

In [69]:
df2 = gmx.get_foliations()

In [70]:
df2

,X,Y,Z,dip,azimuth,polarity,series
0,308193.6523,6686450.844,-1509.864258,7.170060,236.1780,true,Basement
1,318784.6184,6649786.250,-1900.061523,4.163730,172.9370,true,Basement
2,345508.9375,6580611.609,-5718.043945,15.139600,95.7105,true,Basement
3,342021.9980,6635382.594,-3511.806641,14.772300,58.7681,true,Basement
4,328581.4648,6711449.891,-4303.744141,13.058800,72.5252,true,Basement
5,344933.2676,6655603.156,-7685.739258,7.034370,131.1860,true,Basement
6,396977.6484,6592402.875,-5706.252930,4.737500,215.4070,true,Basement
7,390009.6797,6656927.625,-6895.182617,4.919520,180.7740,true,Basement
8,386350.1094,6700971.625,-1114.322510,6.124490,173.6600,true,Basement
9,372326.3164,6583774.734,-11292.835940,6.342510,62.3241,true,Basement


In [53]:
fmts = [c.attrib["Name"] for c in gmx.root[5][1]]

In [54]:
fmts

['Basement',
 'Cattamarra_Coal_Measures',
 'Kockatea_Shale',
 'Late_Permian',
 'Lesueur_Ss',
 'Neocomian_Unc',
 'Sea_level',
 'Topo_and_bathy',
 'Yarragadee_Fm',
 'Moho',
 'Lower_crust']

In [61]:
i = 0
['X', 'Y', 'Z', 'dip', 'azimuth', 'polarity', 'series']

fol = []
for i, s in enumerate(gmx.root[5][1]):
    for c in gmx.root[5][1][i]:
        entry = c[0][0].text.split(",")
        entry.append(c.get("Dip"))
        entry.append(c.get("Azimuth"))
        entry.append(c.get("NormalPolarity"))
        entry.append(s.get("Name"))
    
        fol.append(entry)

In [62]:
fol

[['308193.6523',
  '6686450.844',
  '-1509.864258',
  '7.17006',
  '236.178',
  'true',
  'Basement'],
 ['318784.6184',
  '6649786.25',
  '-1900.061523',
  '4.16373',
  '172.937',
  'true',
  'Basement'],
 ['345508.9375',
  '6580611.609',
  '-5718.043945',
  '15.1396',
  '95.7105',
  'true',
  'Basement'],
 ['342021.998',
  '6635382.594',
  '-3511.806641',
  '14.7723',
  '58.7681',
  'true',
  'Basement'],
 ['328581.4648',
  '6711449.891',
  '-4303.744141',
  '13.0588',
  '72.5252',
  'true',
  'Basement'],
 ['344933.2676',
  '6655603.156',
  '-7685.739258',
  '7.03437',
  '131.186',
  'true',
  'Basement'],
 ['396977.6484',
  '6592402.875',
  '-5706.25293',
  '4.7375',
  '215.407',
  'true',
  'Basement'],
 ['390009.6797',
  '6656927.625',
  '-6895.182617',
  '4.91952',
  '180.774',
  'true',
  'Basement'],
 ['386350.1094',
  '6700971.625',
  '-1114.32251',
  '6.12449',
  '173.66',
  'true',
  'Basement'],
 ['372326.3164',
  '6583774.734',
  '-11292.83594',
  '6.34251',
  '62.3241',
 